In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import json
pd.options.mode.chained_assignment = None

In [6]:
a = []
# b = []
matrix = []
heatmapMatrix = []

locList = ["Grocery Stores, Supermarkets", "Bakeries", "Fast Food Restaurants", "Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances", "Drug Stores and Pharmacies", "Book Stores", "Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"]

for j in range(len(locList)):
    prob = 1
    a = []
    b = []
    for i in range(len(locList)):
        if i != len(locList)-1:
            num = 0.16 * (i+1) *  np.random.rand()
            indprob = num * prob
            a.append(indprob)
            heatmapMatrix.append([locList[i], locList[j], indprob])
            prob = prob - indprob
        else:
            a.append(prob)
            heatmapMatrix.append([locList[i], locList[j], prob])
    matrix.append(a)
    # heatmapMatrix.append(b)

# print(heatmapMatrix)
probMatrix = pd.DataFrame(matrix, columns = locList, index = locList)
heatmapProbMatrix = pd.DataFrame(heatmapMatrix, columns=["place1", "place2", "prob"])

In [7]:
probMatrix.head()

,"Grocery Stores, Supermarkets",Bakeries,Fast Food Restaurants,"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",Drug Stores and Pharmacies,Book Stores,"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local"
"Grocery Stores, Supermarkets",0.115045,0.026030,0.220135,0.345833,0.004850,0.093178,0.194929
Bakeries,0.100526,0.121658,0.196817,0.067134,0.347668,0.021556,0.144642
Fast Food Restaurants,0.011756,0.002288,0.452715,0.164626,0.287603,0.007522,0.073491
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",0.151071,0.026173,0.317794,0.287804,0.093866,0.018748,0.104544
Drug Stores and Pharmacies,0.130540,0.090246,0.237885,0.126630,0.301797,0.032564,0.080338


In [8]:
heatmapProbMatrix.to_csv("heatmapProbMatrix.csv")

In [9]:
pd.Timestamp("2020-04-30 09:00:00") + pd.offsets.Minute(10)  

Timestamp('2020-04-30 09:10:00')

In [10]:
%%time
allRec = []
routeLenth = 8
start_time = pd.Timestamp("2020-04-30")
transaction_type_list = ["online", "offline"]
random_merchant_list = ["merchantA", "merchantB", "merchantC"]

# for j in range(routeLenth-1):
# #     rec = []
#     name = "people"+str(j)
pplCount = 0
for j in range(300): 
    for num, location in zip(range(len(locList)), locList):
        name = "people"+str(pplCount)
        station = location
        rec = []
        time = pd.Timestamp("2020-04-30 09:00:00") 
        rec.append(name)#name
        rec.append(station)#location
        rec.append(np.random.choice(random_merchant_list, 1)[0])
        rec.append(time) #time
        rec.append(np.random.randint(10, high=80)) #transaction val
        rec.append(np.random.choice(transaction_type_list, 1)[0]) #transaction type
        allRec.append(rec)
        pplCount += 1
        for i in range(routeLenth-1):
            rec = []
            time += pd.offsets.Minute(np.random.randint(1, high=200))  
            # print(station)
            probList = probMatrix.loc[station].to_list()
            station = np.random.choice(locList, 1, p=probList)[0]
            rec.append(name)#name
            rec.append(station)#mcc
            rec.append(np.random.choice(random_merchant_list, 1)[0])
            rec.append(time) #time
            rec.append(np.random.randint(10, high=80)) #transaction val
            rec.append(np.random.choice(transaction_type_list, 1)[0]) #transaction type
            
            allRec.append(rec)



CPU times: user 4.04 s, sys: 8.09 ms, total: 4.05 s
Wall time: 4.05 s


In [14]:
records = pd.DataFrame(allRec, columns = ["name", "mcc", "merchant", "time", "transaction_value", "transaction_type"])

In [15]:
records.head()

,name,mcc,merchant,time,transaction_value,transaction_type
0,people0,"Grocery Stores, Supermarkets",merchantC,2020-04-30 09:00:00,66,online
1,people0,"Furniture, Home Furnishings, and Equipment Sto...",merchantA,2020-04-30 10:20:00,45,online
2,people0,Fast Food Restaurants,merchantC,2020-04-30 11:08:00,32,offline
3,people0,Fast Food Restaurants,merchantA,2020-04-30 13:57:00,65,online
4,people0,Fast Food Restaurants,merchantC,2020-04-30 16:30:00,30,offline


In [16]:
len(records)

16800

In [2]:
%%time
records["name"] = records["name"].astype("category")
records["mcc"] = records["mcc"].astype("category")
nameList = records["name"].unique()
record_with_tinterval = pd.DataFrame([], columns = ["name", "mcc", "merchant", "time", "transaction_value", "transaction_type", "time_interval_to_next", "time_interval_from_last"])
#calculate time interval
for name in nameList:
    record_by_name = records[records["name"] == name]
    record_by_name["time_interval_to_next"] = -record_by_name["time"].diff(periods = -1)
    record_by_name["time_interval_from_last"] = record_by_name["time"].diff(periods = 1)
    record_by_name["time_interval_to_next"]
    record_by_name.replace(pd.Timedelta('nat'), pd.Timedelta(0), inplace=True)
    
    record_with_tinterval = record_with_tinterval.append(record_by_name)
#     print(record_by_name.head(2), record_with_tinterval.head())

record_with_tinterval.head()

NameError: name 'records' is not defined

In [15]:
record_with_tinterval.to_csv("data_example_short_with_type.csv")

# Here is the main code

In [2]:
%%time
#read data and change data type
record_with_tinterval = pd.read_csv("data_example_short_with_type.csv", index_col = 0)

# record_with_tinterval["name"] = record_with_tinterval["name"].astype("category")
# record_with_tinterval["mcc"] = record_with_tinterval["mcc"].astype("category")
# record_with_tinterval["transaction_type"] = record_with_tinterval["transaction_type"].astype("category")
record_with_tinterval["time"] =  pd.to_datetime(record_with_tinterval["time"])
record_with_tinterval["time_interval_to_next"] =  pd.to_timedelta(record_with_tinterval["time_interval_to_next"])
record_with_tinterval["time_interval_from_last"] =  pd.to_timedelta(record_with_tinterval["time_interval_from_last"])


CPU times: user 116 ms, sys: 5.64 ms, total: 122 ms
Wall time: 122 ms


In [3]:
%%time
record_with_tinterval_cpy = record_with_tinterval.copy()
# make a "next station" column
record_with_tinterval_cpy["next_station"] = record_with_tinterval_cpy["mcc"].shift(-1)
# delete every ppl's last "next_station" since it is another one's first one
record_with_tinterval_cpy.loc[record_with_tinterval_cpy.groupby("name").tail(1).index, "next_station"] = np.nan
nxt_station_g = record_with_tinterval_cpy[["mcc", "next_station", "name"]].dropna().groupby(["mcc", "next_station"]).count().reset_index().rename(columns = {"name": "count"})
nxt_station_allcnt = nxt_station_g.groupby("mcc").agg({"count":"sum"}).reset_index()
nxt_station_prob = nxt_station_g.merge(nxt_station_allcnt, how = "outer", on = "mcc")
nxt_station_prob["prob"] = nxt_station_prob["count_x"]/nxt_station_prob["count_y"]
nxt_station_prob.rename(columns = {"mcc":"place1", "next_station":"place2"},inplace=True)
nxt_station_prob.drop(columns = ["count_x", "count_y"], inplace = True)
nxt_station_allcnt 


CPU times: user 31.1 ms, sys: 3.96 ms, total: 35 ms
Wall time: 37.5 ms


,mcc,count
0,Bakeries,1761
1,Book Stores,1534
2,Drug Stores and Pharmacies,3395
3,Fast Food Restaurants,2233
4,"Furniture, Home Furnishings, and Equipment Sto...",2107
5,"Grocery Stores, Supermarkets",1751
6,"Motor Freight Carriers, Moving and Storage Com...",1919


In [4]:
record_with_tinterval.dtypes

name                                object
mcc                                 object
merchant                            object
time                        datetime64[ns]
transaction_value                    int64
transaction_type                    object
time_interval_to_next      timedelta64[ns]
time_interval_from_last    timedelta64[ns]
dtype: object

In [5]:
#apply time filter: start time -> end time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
test_time_range_filtered = record_with_tinterval[(record_with_tinterval["time"] > startTime)
                      & (record_with_tinterval["time"] < endTime)]

In [6]:
%%time
#like SQL's row_numebr, give a rank as time

test_time_range_filtered["step"] = test_time_range_filtered.groupby(["name"])["time"].rank().astype(int)

CPU times: user 6.28 ms, sys: 984 µs, total: 7.26 ms
Wall time: 6.25 ms


In [7]:
test_time_range_filtered.head()

,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,1
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,2
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,3
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,4
5,people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,5


In [8]:
len(test_time_range_filtered)

9802

In [9]:
%%time
unique_length = pd.DataFrame(columns = ["pack_length"])
unique_length

CPU times: user 2.4 ms, sys: 52 µs, total: 2.45 ms
Wall time: 2.42 ms


,pack_length


In [10]:
%%time
allList = [["Book Stores", "Fast Food Restaurants"],["Fast Food Restaurants", "Book Stores"]]
left_alignment_group = []
unique_length = pd.DataFrame(columns = ["pack_length"])
all_index = np.array([])
test_time_range_filtered_cpy = test_time_range_filtered.copy()
for j in range(len(allList)):
#     print(j)
    test_time_range_filtered_cpy = test_time_range_filtered_cpy[~test_time_range_filtered_cpy.isin(all_index)]
    #define kernel
    kernel = []
    id_list = []
    queryList = allList[j]
    for i in range(len(queryList)):
        if i == 0:
            mcc_id_transform = (test_time_range_filtered_cpy["mcc"]==queryList[i])*(i+1)
        else:
            mcc_id_transform +=(test_time_range_filtered_cpy["mcc"]==queryList[i])*(i+1)
        kernel.append((i+1)*(i+1)) # kernel = id^2
        id_list.append(i+1)

    test_time_range_filtered_cpy["transform"] = mcc_id_transform
    id_list.reverse()
    kernel.reverse()
    # so if id = [1,2], kernel = [1,4]
    # filter all the last step in the querylist by matching convolve result. eg. two mccs -> [1,2] dot [1,4] = 9
    test_time_range_filtered_cpy["pattern"] = list(np.convolve(mcc_id_transform, kernel,"full") == np.dot(id_list,kernel))[:-len(queryList)+1] # as using full mode, the convolve return would be longer than the df
    packed_data_query = test_time_range_filtered_cpy[(test_time_range_filtered_cpy["pattern"]) & (test_time_range_filtered_cpy["step"]>len(queryList)-1)] # avoid all the 
    grouped_packed_data_query = packed_data_query.groupby("name")
    step_group = grouped_packed_data_query["step"].min().dropna()
    left_alignment_pos_this = step_group.max()
    left_alignment_group.append(left_alignment_pos_this)
    # record length
    unique_this = pd.DataFrame(grouped_packed_data_query["step"].min().dropna())
    unique_this["pack_length"] = len(queryList)
    
    # 
    packed_data_query_route_this = test_time_range_filtered[test_time_range_filtered["name"].isin(step_group.index)]
    packed_data_query_route_this.set_index("name", inplace = True)
    alignment_offset = left_alignment_pos_this - grouped_packed_data_query["step"].min().dropna()
    packed_data_query_route_this["offset"] = alignment_offset
    packed_data_query_route_this["alignment_pos"] = left_alignment_pos_this
#     print(j)
    if j == 0:
        unique_length = unique_this
        all_index = step_group.index
        packed_data_query_route = packed_data_query_route_this
    else:
        unique_length = unique_length.append(unique_this)
        this_index = step_group.index
        all_index.append(this_index)
        packed_data_query_route = packed_data_query_route.append(packed_data_query_route_this)
    
# len(allList)

CPU times: user 345 ms, sys: 4.61 ms, total: 350 ms
Wall time: 349 ms


In [342]:
# find max alignment position
max_alignment_pos = packed_data_query_route["alignment_pos"].max()
packed_data_query_route["alignment_pos_max"] = max_alignment_pos
packed_data_query_route["offset"] += (packed_data_query_route["alignment_pos_max"] - packed_data_query_route["alignment_pos"])

In [343]:
# take max left alignment pos
# packed_data_query_route.set_index("name", inplace = True)
packed_data_query_route["step"] += packed_data_query_route["offset"]
packed_data_query_route.drop(columns = ["offset", "alignment_pos", "alignment_pos_max"], inplace= True)
shaped_routes_packed = packed_data_query_route.pivot(index = packed_data_query_route.index, columns =  "step")

shaped_routes_packed.head()

mcc                                        \
step         1    2    3                           4    
name                                                    
people1000  NaN  NaN  NaN                         NaN   
people1002  NaN  NaN  NaN  Drug Stores and Pharmacies   
people103   NaN  NaN  NaN                         NaN   
people1039  NaN  NaN  NaN                         NaN   
people1048  NaN  NaN  NaN                         NaN   

                                                               \
step                                                       5    
name                                                            
people1000                                                NaN   
people1002                                           Bakeries   
people103   Furniture, Home Furnishings, and Equipment Sto...   
people1039  Furniture, Home Furnishings, and Equipment Sto...   
people1048                       Grocery Stores, Supermarkets   

                                                          \
step                           6                      7    
name                                                       
people1000            Book Stores  Fast Food Restaurants   
people1002  Fast Food Restaurants            Book Stores   
people103             Book Stores  Fast Food Restaurants   
people1039  Fast Food Restaurants            Book Stores   
people1048            Book Stores  Fast Food Restaurants   

                                        \
step                                8    
name                                     
people1000                 Book Stores   
people1002                         NaN   
people103        Fast Food Restaurants   
people1039                         NaN   
people1048  Drug Stores and Pharmacies   

                                                                    ...  \
step                                                       9    10  ...   
name                                                                ...   
people1000  Furniture, Home Furnishings, and Equipment Sto...  NaN  ...   
people1002                                                NaN  NaN  ...   
people103                          Drug Stores and Pharmacies  NaN  ...   
people1039                                                NaN  NaN  ...   
people1048                                                NaN  NaN  ...   

           time_interval_from_last                                      \
step                            3        4        5        6        7    
name                                                                     
people1000                     NaT      NaT      NaT 03:00:00 02:41:00   
people1002                     NaT 03:16:00 02:12:00 00:26:00 01:44:00   
people103                      NaT      NaT 02:43:00 00:43:00 02:18:00   
people1039                     NaT      NaT 03:10:00 02:33:00 02:47:00   
people1048                     NaT      NaT 01:13:00 03:18:00 01:43:00   

                                          
step             8        9   10  11  12  
name                                      
people1000 02:07:00 00:43:00 NaT NaT NaT  
people1002      NaT      NaT NaT NaT NaT  
people103  00:05:00 02:23:00 NaT NaT NaT  
people1039      NaT      NaT NaT NaT NaT  
people1048 01:23:00      NaT NaT NaT NaT  

[5 rows x 84 columns]

In [344]:
shaped_routes_packed["pack_length"] = unique_length["pack_length"]
shaped_routes_packed.head()

mcc                                        \
step          1    2    3                           4   
name                                                    
people1000  NaN  NaN  NaN                         NaN   
people1002  NaN  NaN  NaN  Drug Stores and Pharmacies   
people103   NaN  NaN  NaN                         NaN   
people1039  NaN  NaN  NaN                         NaN   
people1048  NaN  NaN  NaN                         NaN   

                                                               \
step                                                        5   
name                                                            
people1000                                                NaN   
people1002                                           Bakeries   
people103   Furniture, Home Furnishings, and Equipment Sto...   
people1039  Furniture, Home Furnishings, and Equipment Sto...   
people1048                       Grocery Stores, Supermarkets   

                                                          \
step                            6                      7   
name                                                       
people1000            Book Stores  Fast Food Restaurants   
people1002  Fast Food Restaurants            Book Stores   
people103             Book Stores  Fast Food Restaurants   
people1039  Fast Food Restaurants            Book Stores   
people1048            Book Stores  Fast Food Restaurants   

                                        \
step                                 8   
name                                     
people1000                 Book Stores   
people1002                         NaN   
people103        Fast Food Restaurants   
people1039                         NaN   
people1048  Drug Stores and Pharmacies   

                                                                    ...  \
step                                                        9   10  ...   
name                                                                ...   
people1000  Furniture, Home Furnishings, and Equipment Sto...  NaN  ...   
people1002                                                NaN  NaN  ...   
people103                          Drug Stores and Pharmacies  NaN  ...   
people1039                                                NaN  NaN  ...   
people1048                                                NaN  NaN  ...   

           time_interval_from_last                                      \
step                             4        5        6        7        8   
name                                                                     
people1000                     NaT      NaT 03:00:00 02:41:00 02:07:00   
people1002                03:16:00 02:12:00 00:26:00 01:44:00      NaT   
people103                      NaT 02:43:00 00:43:00 02:18:00 00:05:00   
people1039                     NaT 03:10:00 02:33:00 02:47:00      NaT   
people1048                     NaT 01:13:00 03:18:00 01:43:00 01:23:00   

                                pack_length  
step              9  10  11  12              
name                                         
people1000 00:43:00 NaT NaT NaT           2  
people1002      NaT NaT NaT NaT           2  
people103  02:23:00 NaT NaT NaT           2  
people1039      NaT NaT NaT NaT           2  
people1048      NaT NaT NaT NaT           2  

[5 rows x 85 columns]

In [345]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(100)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = -1
#calculate step pos in the data
raw_step = single_sequence + max_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 6


if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes_packed["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes_packed["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
    # add transaction value column
    links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
    links["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
    links["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
elif single_sequence < 0:
    #when go left, because the alignment is conducted with the last stop in the query list
    counter = 0
    for pack_length in shaped_routes_packed["pack_length"].unique():

        source_step = raw_step+1-(pack_length-1)
        target_step = raw_step-(pack_length-1)
        shaped_this_pack = shaped_routes_packed[shaped_routes_packed["pack_length"] == pack_length]
        filtered_routes_this = shaped_this_pack[(shaped_this_pack["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_this_pack["time_interval_from_last"][source_step] < time_interval_limit)]
        links_this = filtered_routes_this["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
        links_this["transaction_value"] = filtered_routes_this.xs(("transaction_value", target_step), axis=1)
        links_this["transaction_type"] = filtered_routes_this.xs(("transaction_type", target_step), axis=1)
        links_this["merchant"] = filtered_routes_this.xs(("merchant", target_step), axis=1)
        if counter == 0:
            links = links_this
        else:
            links = links.append(links_this)
        counter += 1
#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    print("lll")
    links = links[links.index.isin(last_step_routes)]



# column lists that use later
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
# get nodes grouped
nodes_g = links[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links[columns_transaction_value].groupby(["target"]).groups.values()
# filter: node count >0
nodes_g = nodes_g[nodes_g["count"] > 0].reset_index(drop=True)
# group to deal with transaction type
nodes_transtype_groupby = links[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_transtype = nodes_transtype_groupby.count().dropna()
# get routes from diff transaction types
nodes_transtype["type_routes"] = list(nodes_transtype_groupby.groups.values())
nodes_transtype_cnt = nodes_transtype.unstack()["transaction_value"].reset_index().sort_values(by = ["target"]).reset_index(drop=True)
nodes_transtype_routes = nodes_transtype.unstack()["type_routes"].reset_index().sort_values(by = ["target"]).reset_index(drop=True)
# add trans type count
nodes_g["offline_count"] = nodes_transtype_cnt["offline"]
nodes_g["online_count"] = nodes_transtype_cnt["online"]
# add trans type trajectory id
nodes_g["offline_route"] = nodes_transtype_routes["offline"]
nodes_g["online_route"] = nodes_transtype_routes["online"]
#add atv
nodes_g["atv"] = links.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get largest X results
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
for row in nodes_result.loc[nodes_result["offline_route"].isna(), "offline_route"].index:
    nodes_result.at[row, "offline_route"] = np.array([])
for row in nodes_result.loc[nodes_result["online_route"].isna(), "online_route"].index:
    nodes_result.at[row, "online_route"] = np.array([])
nodes_result["offline_count"].fillna(0, inplace = True)
nodes_result["online_count"].fillna(0, inplace = True)
nodes_result["offline_route"] = nodes_result["offline_route"].apply(lambda x: x.tolist())
nodes_result["online_route"] = nodes_result["online_route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.groupby(["source","target"]).count().rename(columns = {"merchant":"count"}).reset_index().dropna()
# print(links_g)
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links[["source", "target", "transaction_value"]].groupby(["source","target"]).mean().dropna().reset_index()["transaction_value"]
links_transtype_groupby = links[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_transtype = links_transtype_groupby.count().dropna()
# group to deal with transaction type
links_transtype["type_routes"] = list(links_transtype_groupby.groups.values())
links_transtype_cnt = links_transtype.unstack()["transaction_value"].reset_index().sort_values(by = ["source", "target"]).reset_index()
links_transtype_routes = links_transtype.unstack()["type_routes"].reset_index().sort_values(by = ["source", "target"]).reset_index()
# add trans type count
links_g["offline_count"] = links_transtype_cnt["offline"]
links_g["online_count"] = links_transtype_cnt["online"]
# add trans type traj id
links_g["offline_route"] = links_transtype_routes["offline"]
links_g["online_route"] = links_transtype_routes["online"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# print(links_result)
# print(links_result["offline_route"].isna())

# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
#fill null list
for row in links_result.loc[links_result["offline_route"].isna(), "offline_route"].index:
    links_result.at[row, "offline_route"] = np.array([])
for row in links_result.loc[links_result["online_route"].isna(), "online_route"].index:
    links_result.at[row, "online_route"] = np.array([])
links_result["offline_count"].fillna(0, inplace = True)
links_result["online_count"].fillna(0, inplace = True)
links_result["offline_route"] = links_result["offline_route"].apply(lambda x: x.tolist())
links_result["online_route"] = links_result["online_route"].apply(lambda x: x.tolist())

links_result_dict = links_result.to_dict('records')
nodes_result_dict = nodes_result.to_dict('records')

last_step_routes = this_step_routes

CPU times: user 72.6 ms, sys: 1.97 ms, total: 74.6 ms
Wall time: 72.7 ms


In [346]:
# step -1
links_result

step,source,target,transaction_type,count,route,atv,offline_count,online_count,offline_route,online_route
0,Book Stores,Bakeries,6,6,"[people1568, people1645, people1729, people502...",37.166667,5.0,1.0,"[people1568, people1645, people502, people9, p...",[people1729]
1,Book Stores,Drug Stores and Pharmacies,16,16,"[people1089, people1456, people1560, people159...",49.750000,7.0,9.0,"[people1089, people223, people237, people499, ...","[people1456, people1560, people1591, people228..."
2,Book Stores,Fast Food Restaurants,11,11,"[people117, people1294, people1450, people1679...",47.090909,5.0,6.0,"[people1294, people1450, people1679, people202...","[people117, people1727, people1984, people635,..."
3,Book Stores,"Furniture, Home Furnishings, and Equipment Sto...",14,14,"[people103, people1057, people1079, people1766...",46.428571,7.0,7.0,"[people1057, people1079, people1954, people311...","[people103, people1766, people2094, people33, ..."
4,Book Stores,"Grocery Stores, Supermarkets",2,2,"[people540, people858]",52.500000,1.0,1.0,[people540],[people858]
5,Book Stores,"Motor Freight Carriers, Moving and Storage Com...",3,3,"[people1840, people337, people815]",46.000000,1.0,2.0,[people1840],"[people337, people815]"
6,Fast Food Restaurants,Bakeries,9,9,"[people1002, people1060, people1081, people115...",53.000000,6.0,3.0,"[people1060, people1081, people115, people1817...","[people1002, people1829, people1891]"
7,Fast Food Restaurants,Drug Stores and Pharmacies,1,1,[people120],74.000000,0.0,1.0,[],[people120]
8,Fast Food Restaurants,Fast Food Restaurants,7,7,"[people1091, people12, people1332, people1429,...",41.285714,2.0,5.0,"[people12, people457]","[people1091, people1332, people1429, people176..."
9,Fast Food Restaurants,"Furniture, Home Furnishings, and Equipment Sto...",8,8,"[people1065, people144, people1445, people1615...",51.875000,6.0,2.0,"[people1065, people1615, people1650, people171...","[people144, people1445]"


In [347]:
nodes_result

step,target,count,route,offline_count,online_count,offline_route,online_route,atv
0,"Furniture, Home Furnishings, and Equipment Sto...",22,"[people103, people1057, people1065, people1079...",13.0,9.0,"[people1057, people1065, people1079, people161...","[people103, people144, people1445, people1766,...",48.409091
1,Fast Food Restaurants,18,"[people1091, people117, people12, people1294, ...",7.0,11.0,"[people12, people1294, people1450, people1679,...","[people1091, people117, people1332, people1429...",44.833333
2,"Grocery Stores, Supermarkets",18,"[people1123, people1177, people1531, people173...",9.0,9.0,"[people1177, people1531, people1888, people198...","[people1123, people1739, people520, people582,...",45.388889
3,Drug Stores and Pharmacies,17,"[people1089, people120, people1456, people1560...",7.0,10.0,"[people1089, people223, people237, people499, ...","[people120, people1456, people1560, people1591...",51.176471
4,Bakeries,15,"[people1002, people1060, people1081, people115...",11.0,4.0,"[people1060, people1081, people115, people1568...","[people1002, people1729, people1829, people1891]",46.666667
5,"Motor Freight Carriers, Moving and Storage Com...",8,"[people1255, people1441, people1840, people235...",4.0,4.0,"[people1255, people1441, people1840, people873]","[people235, people337, people815, people854]",54.625000


In [340]:
links_result

step,source,target,transaction_type,count,route,atv,offline_count,online_count,offline_route,online_route
0,Book Stores,Bakeries,6,6,"[people1568, people1645, people1729, people502...",37.166667,5.0,1.0,"[people1568, people1645, people502, people9, p...",[people1729]
1,Book Stores,Book Stores,2,2,"[people732, people868]",46.000000,2.0,0.0,"[people732, people868]",[]
2,Book Stores,Drug Stores and Pharmacies,16,16,"[people1089, people1456, people1560, people159...",49.750000,7.0,9.0,"[people1089, people223, people237, people499, ...","[people1456, people1560, people1591, people228..."
3,Book Stores,Fast Food Restaurants,11,11,"[people117, people1294, people1450, people1679...",47.090909,5.0,6.0,"[people1294, people1450, people1679, people202...","[people117, people1727, people1984, people635,..."
4,Book Stores,"Furniture, Home Furnishings, and Equipment Sto...",14,14,"[people103, people1057, people1079, people1766...",46.428571,7.0,7.0,"[people1057, people1079, people1954, people311...","[people103, people1766, people2094, people33, ..."
5,Book Stores,"Motor Freight Carriers, Moving and Storage Com...",3,3,"[people1840, people337, people815]",46.000000,1.0,2.0,[people1840],"[people337, people815]"


In [11]:
%%time
queryMCC = "Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances"
#get largest min "max first position" of the queried MCC in the route as the left alignment position
test_data_query = test_time_range_filtered.loc[test_time_range_filtered["mcc"] == queryMCC]
grouped_test_data_query = test_data_query.groupby("name")
left_alignment_pos = grouped_test_data_query["step"].min().max()
need_alignment = grouped_test_data_query["step"].min().dropna() < left_alignment_pos
alignment_offset = left_alignment_pos - grouped_test_data_query["step"].min().dropna()
len(need_alignment)

CPU times: user 5.94 ms, sys: 1.15 ms, total: 7.1 ms
Wall time: 6.04 ms


1097

In [12]:
%%time
test_data_query_route = test_time_range_filtered[test_time_range_filtered["name"].isin(need_alignment.index)]
test_data_query_route.head()

CPU times: user 2.59 ms, sys: 929 µs, total: 3.52 ms
Wall time: 2.67 ms


,name,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step
1,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,1
2,people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,2
3,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,3
4,people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,4
5,people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,5


In [13]:
test_data_query_route.shape

(5308, 9)

In [14]:
%%time
#move the order acoording to offset
test_data_query_route.set_index("name", inplace = True)
test_data_query_route["offset"] = alignment_offset
test_data_query_route["step"] = test_data_query_route["step"] + test_data_query_route["offset"]
test_data_query_route.head()

CPU times: user 2.89 ms, sys: 170 µs, total: 3.06 ms
Wall time: 2.9 ms


,mcc,merchant,time,transaction_value,transaction_type,time_interval_to_next,time_interval_from_last,step,offset
name,,,,,,,,,
people0,"Motor Freight Carriers, Moving and Storage Com...",merchantA,2020-04-30 10:12:00,15,online,01:28:00,01:12:00,2,1
people0,"Motor Freight Carriers, Moving and Storage Com...",merchantB,2020-04-30 11:40:00,68,offline,00:56:00,01:28:00,3,1
people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:36:00,40,online,00:18:00,00:56:00,4,1
people0,Drug Stores and Pharmacies,merchantC,2020-04-30 12:54:00,36,online,01:25:00,00:18:00,5,1
people0,Bakeries,merchantB,2020-04-30 14:19:00,70,offline,00:11:00,01:25:00,6,1


In [15]:
%%time
# columns = ["mcc", "transaction_value", "time_interval_to_next", "time_interval_from_last"]
test_data_query_route.drop(columns = ["offset"], inplace= True)
shaped_routes = test_data_query_route.pivot(index = test_data_query_route.index, columns =  "step")
shaped_routes.head()

CPU times: user 18.4 ms, sys: 1.55 ms, total: 19.9 ms
Wall time: 18.8 ms


mcc                                                     \
step         1                                                  2    
name                                                                 
people0     NaN  Motor Freight Carriers, Moving and Storage Com...   
people1     NaN                                                NaN   
people100   NaN                                                NaN   
people1000  NaN                                                NaN   
people1003  NaN                                                NaN   

                                                               \
step                                                       3    
name                                                            
people0     Motor Freight Carriers, Moving and Storage Com...   
people1                                                   NaN   
people100                                                 NaN   
people1000                                                NaN   
people1003                                                NaN   

                                                                    \
step                                4                           5    
name                                                                 
people0     Drug Stores and Pharmacies  Drug Stores and Pharmacies   
people1          Fast Food Restaurants  Drug Stores and Pharmacies   
people100                          NaN                         NaN   
people1000                 Book Stores       Fast Food Restaurants   
people1003                         NaN                         NaN   

                                                                            \
step                 6                                                  7    
name                                                                         
people0        Bakeries  Furniture, Home Furnishings, and Equipment Sto...   
people1        Bakeries  Furniture, Home Furnishings, and Equipment Sto...   
people100           NaN  Furniture, Home Furnishings, and Equipment Sto...   
people1000  Book Stores  Furniture, Home Furnishings, and Equipment Sto...   
people1003          NaN  Furniture, Home Furnishings, and Equipment Sto...   

                                   \
step                           8    
name                                
people0     Fast Food Restaurants   
people1                  Bakeries   
people100   Fast Food Restaurants   
people1000                    NaN   
people1003  Fast Food Restaurants   

                                                                            \
step                                                       9            10   
name                                                                         
people0                                                   NaN          NaN   
people1     Furniture, Home Furnishings, and Equipment Sto...          NaN   
people100   Furniture, Home Furnishings, and Equipment Sto...  Book Stores   
people1000                                                NaN          NaN   
people1003  Furniture, Home Furnishings, and Equipment Sto...  Book Stores   

            ... time_interval_from_last                                      \
step        ...                      4        5        6        7        8    
name        ...                                                               
people0     ...                00:56:00 00:18:00 01:25:00 00:11:00 02:13:00   
people1     ...                01:58:00 02:07:00 02:00:00 00:25:00 00:37:00   
people100   ...                     NaT      NaT      NaT 01:23:00 00:47:00   
people1000  ...                03:00:00 02:41:00 02:07:00 00:43:00      NaT   
people1003  ...                     NaT      NaT      NaT 01:24:00 00:18:00   

                                                    
step             9        10       11       12  13  
name                                                
people0         NaT      NaT  

In [29]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(200)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 6


if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
else:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    print("lll")
    links = links[links.index.isin(last_step_routes)]

# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
links["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
# column lists that use later
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
# get nodes grouped
nodes_g = links[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = list(links[columns_transaction_value].groupby(["target"]).groups.values())
# filter: node count >0
nodes_g = nodes_g[nodes_g["count"] > 0].reset_index(drop=True)
# group to deal with transaction type
nodes_transtype_groupby = links[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_transtype = nodes_transtype_groupby.count().dropna()
# get routes from diff transaction types
nodes_transtype["type_routes"] = list(nodes_transtype_groupby.groups.values())
nodes_transtype_cnt = nodes_transtype.unstack()["transaction_value"].reset_index().sort_values(by = ["target"]).reset_index(drop=True)
nodes_transtype_routes = nodes_transtype.unstack()["type_routes"].reset_index().sort_values(by = ["target"]).reset_index(drop=True)
# add trans type count
nodes_g["offline_count"] = nodes_transtype_cnt["offline"]
nodes_g["online_count"] = nodes_transtype_cnt["online"]
# add trans type trajectory id
nodes_g["offline_route"] = nodes_transtype_routes["offline"]
nodes_g["online_route"] = nodes_transtype_routes["online"]
#add atv
nodes_g["atv"] = links.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get largest X results
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
for row in nodes_result.loc[nodes_result["offline_route"].isna(), "offline_route"].index:
    nodes_result.at[row, "offline_route"] = np.array([])
for row in nodes_result.loc[nodes_result["online_route"].isna(), "online_route"].index:
    nodes_result.at[row, "online_route"] = np.array([])
nodes_result["offline_count"].fillna(0, inplace = True)
nodes_result["online_count"].fillna(0, inplace = True)
nodes_result["offline_route"] = nodes_result["offline_route"].apply(lambda x: x.tolist())
nodes_result["online_route"] = nodes_result["online_route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.groupby(["source","target"]).count().rename(columns = {"merchant":"count"}).reset_index().dropna()
# print(links_g)
# capture links' routes by people's identifier/name
links_g["route"] = list(links.groupby(["source","target"]).groups.values())
# add atv column
links_g["atv"] = links[["source", "target", "transaction_value"]].groupby(["source","target"]).mean().dropna().reset_index()["transaction_value"]
links_transtype_groupby = links[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_transtype = links_transtype_groupby.count().dropna()
# group to deal with transaction type
links_transtype["type_routes"] = list(links_transtype_groupby.groups.values())
links_transtype_cnt = links_transtype.unstack()["transaction_value"].reset_index().sort_values(by = ["source", "target"]).reset_index()
links_transtype_routes = links_transtype.unstack()["type_routes"].reset_index().sort_values(by = ["source", "target"]).reset_index()
# add trans type count
links_g["offline_count"] = links_transtype_cnt["offline"]
links_g["online_count"] = links_transtype_cnt["online"]
# add trans type traj id
links_g["offline_route"] = links_transtype_routes["offline"]
links_g["online_route"] = links_transtype_routes["online"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# print(links_result)
# print(links_result["offline_route"].isna())

# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
#fill null list
for row in links_result.loc[links_result["offline_route"].isna(), "offline_route"].index:
    links_result.at[row, "offline_route"] = np.array([])
for row in links_result.loc[links_result["online_route"].isna(), "online_route"].index:
    links_result.at[row, "online_route"] = np.array([])
links_result["offline_count"].fillna(0, inplace = True)
links_result["online_count"].fillna(0, inplace = True)
links_result["offline_route"] = links_result["offline_route"].apply(lambda x: x.tolist())
links_result["online_route"] = links_result["online_route"].apply(lambda x: x.tolist())

links_result_dict = links_result.to_dict('records')
nodes_result_dict = nodes_result.to_dict('records')

last_step_routes = this_step_routes

CPU times: user 80.8 ms, sys: 1.69 ms, total: 82.5 ms
Wall time: 81.3 ms


In [28]:
nodes_result

step,target,count,route,offline_count,online_count,offline_route,online_route,atv
0,"Furniture, Home Furnishings, and Equipment Sto...",913,"[people0, people1, people100, people1003, peop...",456,457,"[people100, people1007, people1017, people1020...","[people0, people1, people1003, people1006, peo...",44.191676


In [30]:
nodes_result

step,target,count,route,offline_count,online_count,offline_route,online_route,atv
0,Drug Stores and Pharmacies,223,"[people1006, people102, people1023, people1025...",117,106,"[people1006, people1025, people1032, people104...","[people102, people1023, people1037, people1071...",46.479821
1,Fast Food Restaurants,177,"[people0, people100, people1003, people1020, p...",87,90,"[people1039, people1061, people1065, people110...","[people0, people100, people1003, people1020, p...",43.824859
2,Book Stores,172,"[people1007, people1009, people1028, people103...",98,74,"[people1007, people1009, people1028, people103...","[people1084, people1094, people1096, people111...",46.325581
3,"Motor Freight Carriers, Moving and Storage Com...",116,"[people1093, people1097, people1130, people118...",61,55,"[people1202, people1244, people1265, people132...","[people1093, people1097, people1130, people118...",42.586207
4,Bakeries,113,"[people1, people1017, people1024, people1067, ...",54,59,"[people1067, people1068, people1082, people115...","[people1, people1017, people1024, people1076, ...",45.796460
5,"Grocery Stores, Supermarkets",66,"[people109, people1099, people1157, people1160...",34,32,"[people109, people1157, people1160, people1201...","[people1099, people1196, people1252, people140...",42.106061


In [ ]:
this = "people316"

In [81]:
list(links_result[links_result["target"] == "Drug Stores and Pharmacies"]["route"])

[['people1012',
  'people1074',
  'people1100',
  'people1135',
  'people1148',
  'people1180',
  'people1200',
  'people1201',
  'people1246',
  'people1278',
  'people1325',
  'people1362',
  'people1364',
  'people1408',
  'people1423',
  'people1454',
  'people1467',
  'people1475',
  'people1528',
  'people1549',
  'people1611',
  'people164',
  'people1664',
  'people1771',
  'people1781',
  'people1805',
  'people1814',
  'people1859',
  'people1866',
  'people1867',
  'people1907',
  'people1920',
  'people1989',
  'people1993',
  'people200',
  'people2014',
  'people2025',
  'people2027',
  'people2053',
  'people231',
  'people255',
  'people27',
  'people297',
  'people301',
  'people303',
  'people304',
  'people424',
  'people487',
  'people491',
  'people549',
  'people56',
  'people603',
  'people675',
  'people696',
  'people735',
  'people750',
  'people782',
  'people784',
  'people794',
  'people81',
  'people828',
  'people869',
  'people88',
  'people881',
  'peop

In [85]:
links_result

step,source,target,transaction_type,count,route,atv,offline_count,online_count,offline_route,online_route
0,Bakeries,Bakeries,4,4,"[people1166, people1595, people2092, people650]",46.250000,1.0,3.0,[people1595],"[people1166, people2092, people650]"
1,Bakeries,Book Stores,1,1,[people586],76.000000,0.0,1.0,[],[people586]
2,Bakeries,Drug Stores and Pharmacies,28,28,"[people0, people1, people1128, people1197, peo...",49.571429,15.0,13.0,"[people1, people1197, people1507, people151, p...","[people0, people1128, people1217, people1291, ..."
3,Bakeries,"Grocery Stores, Supermarkets",10,10,"[people1099, people1393, people1395, people150...",43.900000,4.0,6.0,"[people1099, people1395, people1500, people42]","[people1393, people1636, people1646, people203..."
4,Bakeries,"Motor Freight Carriers, Moving and Storage Com...",10,10,"[people1380, people1427, people1443, people18,...",48.800000,6.0,4.0,"[people1427, people1443, people18, people275, ...","[people1380, people181, people428, people715]"
5,Book Stores,Bakeries,9,9,"[people1264, people1436, people1822, people206...",42.000000,3.0,6.0,"[people1436, people1822, people667]","[people1264, people2060, people290, people701,..."
6,Book Stores,Book Stores,12,12,"[people1202, people1348, people1531, people159...",53.916667,6.0,6.0,"[people1202, people1348, people1597, people165...","[people1531, people16, people1778, people1878,..."
7,Book Stores,Drug Stores and Pharmacies,52,52,"[people1032, people1104, people1299, people136...",49.076923,31.0,21.0,"[people1104, people1366, people1407, people146...","[people1032, people1299, people1442, people145..."
8,Book Stores,Fast Food Restaurants,53,53,"[people1000, people1071, people1108, people111...",42.849057,28.0,25.0,"[people1000, people1111, people1123, people118...","[people1071, people1108, people115, people1251..."
9,Book Stores,"Grocery Stores, Supermarkets",20,20,"[people1020, people1093, people114, people1219...",40.000000,8.0,12.0,"[people1020, people1093, people1509, people162...","[people114, people1219, people1275, people1640..."


In [91]:
links.loc["people316"]

step
source                 Drug Stores and Pharmacies
target               Grocery Stores, Supermarkets
transaction_value                              64
transaction_type                          offline
merchant                                merchantA
Name: people316, dtype: object

In [107]:
this = "people1310"
this in last_step_routes

False

In [20]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(100)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 5


if single_sequence > 0:
    source_step = raw_step-1
    target_step = raw_step
    #filter the data by time interval
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][source_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][target_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})
elif single_sequence < 0:
    source_step = raw_step+1
    target_step = raw_step
    filtered_routes = shaped_routes[(shaped_routes["time_interval_to_next"][target_step] < time_interval_limit) & (shaped_routes["time_interval_from_last"][source_step] < time_interval_limit)]
    links = filtered_routes["mcc"][[source_step,target_step]].rename(columns = {source_step:"source", target_step:"target"})

#filter links into last step routes, then all the sources should be same as last steps' target
if abs(single_sequence) > 1:
    links = links[links.index.isin(this_step_routes)]
    
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
# get nodes grouped
nodes_g = links.drop(columns = ["source"]).groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.drop(columns = ["source"]).groupby(["target"]).groups.values()
# get top nodes as output result
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# avoid 0 count nodes
nodes_result = nodes_result[nodes_result["count"] > 0]
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())

CPU times: user 42.9 ms, sys: 2.11 ms, total: 45 ms
Wall time: 43.2 ms


In [56]:
links_transtype

step                                                                                              transaction_value  \
source                                             target                       transaction_type                      
Bakeries                                           Bakeries                     offline                           2   
                                                                                online                            3   
                                                   Book Stores                  online                            1   
                                                   Drug Stores and Pharmacies   offline                          12   
                                                                                online                           14   
                                                   Fast Food Restaurants        offline                           6   
                                                                                online                            7   
                                                   Grocery Stores, Supermarkets offline                           3   
                                                                                online                            1   
Drug Stores and Pharmacies                         Bakeries                     offline                          33   
                                                                                online                           21   
                                                   Book Stores                  offline                          10   
                                                                                online                           17   
                                                   Drug Stores and Pharmacies   offline                          27   
                                                                                online                           31   
                                                   Fast Food Restaurants        offline                           8   
                                                                                online                            2   
                                                   Grocery Stores, Supermarkets offline                          17   
                                                                                online                           15   
Fast Food Restaurants                              Book Stores                  offline                           9   
                                                                                online                           10   
                                                   Drug Stores and Pharmacies   offline                          17   
                                                                                online                           19   
                                                   Fast Food Restaurants        offline                          10   
                                                                                online                            8   
                                                   Grocery Stores, Supermarkets offline                          11   
                                                                                online                            7   
Furniture, Home Furnishings, and Equipment Stor... Bakeries                     offline                           1   
                                                                                online                            4   
                                                   Book Stores                  offline                           8   
                                                                                online                            6   
                                                   Drug Stores and Pharma

In [29]:
%%time
#time interval filter
time_interval_limit = pd.offsets.Minute(200)
#set the query center("Bakeries") as step 0(single_sequence), so the steps before are with minus numbers and steps after with positive nums.
#here, single_sequence = 1, which means the next station people go after they go to "Bakeries"
single_sequence = 1
#calculate step pos in the data
raw_step = single_sequence + left_alignment_pos
#the max number of nodes(MCCs) shown in a single step in the visulization
keep_rank_num = 5

CPU times: user 15 µs, sys: 1e+03 ns, total: 16 µs
Wall time: 18.1 µs


In [42]:
# add transaction value column
links["transaction_value"] = filtered_routes.xs(("transaction_value", target_step), axis=1)
links["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
# column lists that use later
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
# get nodes grouped
nodes_g = links[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_value":"count"}).reset_index()
# capture nodes' route
nodes_g["route"] = links.groupby(["target"]).groups.values()
# filter: node count >0
nodes_g = nodes_g[nodes_g["count"] > 0].reset_index(drop=True)
# group to deal with transaction type
nodes_transtype_groupby = links[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_transtype = nodes_transtype_groupby.count().dropna()
# get routes from diff transaction types
nodes_transtype["type_routes"] = nodes_transtype_groupby.groups.values()
nodes_transtype_cnt = nodes_transtype.unstack()["transaction_value"].reset_index(drop=True)
nodes_transtype_routes = nodes_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
nodes_g["offline_count"] = nodes_transtype_cnt["offline"]
nodes_g["online_count"] = nodes_transtype_cnt["online"]
# add trans type trajectory id
nodes_g["offline_route"] = nodes_transtype_routes["offline"]
nodes_g["online_route"] = nodes_transtype_routes["online"]
#add atv
nodes_g["atv"] = links.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get largest X results
nodes_result = nodes_g.nlargest(keep_rank_num, "count").reset_index(drop=True)
# convert array to list - for jsonify
nodes_result["route"] = nodes_result["route"].apply(lambda x: x.tolist())
nodes_result["offline_route"] = nodes_result["offline_route"].apply(lambda x: x.tolist())
nodes_result["online_route"] = nodes_result["online_route"].apply(lambda x: x.tolist())
# get routes of top nodes 
this_step_routes = []
for routes in nodes_result["route"]:
    this_step_routes += routes
# get all links in this step's route
links = links[links.index.isin(this_step_routes)]
# get links grouped into source + target + count format df
links_g = links.reset_index().groupby(["source","target"]).count().dropna().rename(columns = {"name":"count"}).reset_index()
# capture links' routes by people's identifier/name
links_g["route"] = links.groupby(["source","target"]).groups.values()
# add atv column
links_g["atv"] = links[["source", "target", "transaction_value"]].groupby(["source","target"]).mean().dropna().reset_index()["transaction_value"]
links_transtype_groupby = links[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_transtype = links_transtype_groupby.count().dropna()
# group to deal with transaction type
links_transtype["type_routes"] = links_transtype_groupby.groups.values()
links_transtype_cnt = links_transtype.unstack()["transaction_value"].reset_index(drop=True)
links_transtype_routes = links_transtype.unstack()["type_routes"].reset_index(drop=True)
# add trans type count
links_g["offline_count"] = links_transtype_cnt["offline"]
links_g["online_count"] = links_transtype_cnt["online"]
# add trans type traj id
links_g["offline_route"] = links_transtype_routes["offline"]
links_g["online_route"] = links_transtype_routes["online"]
# generate results
links_result = links_g.reset_index(drop = True).drop(columns = ["transaction_value"])
# convert array to list
links_result["route"] = links_result["route"].apply(lambda x: x.tolist())
links_result["offline_route"] = links_result["offline_route"].apply(lambda x: x.tolist())
links_result["online_route"] = links_result["online_route"].apply(lambda x: x.tolist())

In [43]:
links_result

step,source,target,count,transaction_type,merchant,route,atv,offline_count,online_count,offline_route,online_route
0,Bakeries,Bakeries,71.0,71.0,71.0,"[people1025, people1038, people1040, people108...",41.225352,31.0,40.0,"[people1179, people1182, people1207, people121...","[people1025, people1038, people1040, people108..."
1,Bakeries,Drug Stores and Pharmacies,319.0,319.0,319.0,"[people1012, people1014, people1017, people101...",43.946708,162.0,157.0,"[people1012, people1017, people1019, people102...","[people1014, people1024, people1036, people106..."
2,Bakeries,Fast Food Restaurants,137.0,137.0,137.0,"[people1002, people1015, people1021, people103...",45.116788,71.0,66.0,"[people1021, people1032, people1089, people115...","[people1002, people1015, people1060, people106..."
3,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",81.0,81.0,81.0,"[people0, people1, people1006, people1037, peo...",45.913580,35.0,46.0,"[people1037, people1197, people1217, people126...","[people0, people1, people1006, people1099, peo..."
4,Bakeries,"Grocery Stores, Supermarkets",58.0,58.0,58.0,"[people1016, people1049, people1093, people11,...",41.086207,31.0,27.0,"[people1093, people1116, people1151, people130...","[people1016, people1049, people11, people1203,..."


In [26]:
nodes_result

step,target,count,route,offline_count,online_count,offline_route,online_route,atv
0,Drug Stores and Pharmacies,319,"[people1012, people1014, people1017, people101...",162,157,"[people1012, people1017, people1019, people102...","[people1014, people1024, people1036, people106...",43.946708
1,Fast Food Restaurants,137,"[people1002, people1015, people1021, people103...",71,66,"[people1021, people1032, people1089, people115...","[people1002, people1015, people1060, people106...",45.116788
2,"Furniture, Home Furnishings, and Equipment Sto...",81,"[people0, people1, people1006, people1037, peo...",35,46,"[people1037, people1197, people1217, people126...","[people0, people1, people1006, people1099, peo...",45.913580
3,Bakeries,71,"[people1025, people1038, people1040, people108...",31,40,"[people1179, people1182, people1207, people121...","[people1025, people1038, people1040, people108...",41.225352
4,"Grocery Stores, Supermarkets",58,"[people1016, people1049, people1093, people11,...",31,27,"[people1093, people1116, people1151, people130...","[people1016, people1049, people11, people1203,...",41.086207


In [139]:
%%time
links_cpy = links.copy()
links_cpy["transaction_type"] = filtered_routes.xs(("transaction_type", target_step), axis=1)
links_cpy["merchant"] = filtered_routes.xs(("merchant", target_step), axis=1)
links_cpy.head()

CPU times: user 3.32 ms, sys: 941 µs, total: 4.26 ms
Wall time: 3.4 ms


step,source,target,transaction_value,transaction_type,merchant
name,,,,,
people0,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",74.0,online,merchantB
people1,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",77.0,online,merchantA
people1002,Bakeries,Fast Food Restaurants,34.0,online,merchantC
people1006,Bakeries,"Furniture, Home Furnishings, and Equipment Sto...",37.0,online,merchantC
people1012,Bakeries,Drug Stores and Pharmacies,41.0,offline,merchantB


In [23]:
columns_transaction_type = ["source", "target", "transaction_type", "transaction_value"]
columns_transaction_value = ["target", "transaction_value"]
columns_merchant = ["target", "transaction_type", "merchant"]

In [22]:
%%time

# get nodes grouped
nodes_cpy_g = links_cpy[columns_transaction_value].groupby(["target"]).count().rename(columns = {"transaction_type":"count"}).reset_index()

# capture nodes' route
# nodes_cpy_g.sort_values(by = "count", ascending = False, inplace=True)
nodes_cpy_g["route"] = links_cpy.groupby(["target"]).groups.values()
nodes_cpy_g = nodes_cpy_g[nodes_cpy_g["count"] > 0].reset_index(drop=True)
nodes_transtype_groupby = links_cpy[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"])
nodes_cpy_transtype = nodes_transtype_groupby.count().dropna()
nodes_cpy_transtype["type_routes"] = nodes_transtype_groupby.groups.values()
nodes_cpy_transtype_cnt = nodes_cpy_transtype.unstack()["transaction_value"].reset_index(drop=True)
nodes_cpy_transtype_routes = nodes_cpy_transtype.unstack()["type_routes"].reset_index(drop=True)
nodes_cpy_g["offline_count"] = nodes_cpy_transtype_cnt["offline"]
nodes_cpy_g["online_count"] = nodes_cpy_transtype_cnt["online"]
nodes_cpy_g["offline_route"] = nodes_cpy_transtype_routes["offline"]
nodes_cpy_g["online_route"] = nodes_cpy_transtype_routes["online"]
nodes_cpy_g["atv"] = links_cpy.groupby(["target"]).mean().dropna()["transaction_value"].reset_index(drop=True)
# get links grouped into source + target + count format df
links_cpy_g = links_cpy.reset_index().groupby(["source","target"]).count().dropna().rename(columns = {"name":"count"}).reset_index()
# capture links' routes by people's identifier/name
links_cpy_g["route"] = links_cpy.groupby(["source","target"]).groups.values()
# add atv column
links_cpy_g["atv"] = links_cpy.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]
links_transtype_groupby = links_cpy[columns_transaction_type].groupby(by = ["source", "target", "transaction_type"])
links_cpy_transtype = links_transtype_groupby.count().dropna()
links_cpy_transtype["type_routes"] = links_transtype_groupby.groups.values()
links_cpy_transtype_cnt = links_cpy_transtype.unstack()["transaction_value"].reset_index(drop=True)
links_cpy_transtype_routes = links_cpy_transtype.unstack()["type_routes"].reset_index(drop=True)
# links_cpy_transtype["type_routes"] = 
links_cpy_g["offline_count"] = links_cpy_transtype_cnt["offline"]
links_cpy_g["online_count"] = links_cpy_transtype_cnt["online"]
links_cpy_g["offline_route"] = links_cpy_transtype_routes["offline"]
links_cpy_g["online_route"] = links_cpy_transtype_routes["online"]
# links_cpy.groupby(["source","target"]).mean()
# link_cpy_transtype_cnt.reset_index(drop=True)
nodes_cpy_g.nlargest(keep_rank_num+5, "count").reset_index(drop=True)
links_cpy_g

NameError: name 'links_cpy' is not defined

In [259]:
links_cpy[columns_transaction_type].drop(columns = ["source"]).groupby(by = ["target", "transaction_type"]).count().unstack()["transaction_value"].dropna().reset_index(drop=True)["offline"]

0     31.0
1    162.0
2     71.0
3     35.0
4     31.0
Name: offline, dtype: float64

In [150]:
%%time

# nodes_copy = links_copy.drop(columns = ["source"])
# links_copy["target"].value_counts()
# # get links grouped into source + target + count format df
# links_cpy_g = links_cpy.reset_index().groupby(["source","target"]).count().reset_index().dropna().rename(columns = {"name":"count"})
# # capture links' routes by people's identifier/name
# links_cpy_g["route"] = links_cpy.groupby(["source","target"]).groups.values()
# # add atv column
# links_cpy_g["atv"] = links_cpy.groupby(["source","target"]).mean().reset_index().dropna()["transaction_value"]

links_cpy[columns_transaction_type].groupby(by = ["target", "transaction_type"])["transaction_type"].count().unstack()[["offline"]]

CPU times: user 11.9 ms, sys: 1.84 ms, total: 13.7 ms
Wall time: 12.1 ms


transaction_type,offline
target,
Bakeries,31
Book Stores,0
Drug Stores and Pharmacies,162
Fast Food Restaurants,71
"Furniture, Home Furnishings, and Equipment Stores, ExceptAppliances",35
"Grocery Stores, Supermarkets",31
"Motor Freight Carriers, Moving and Storage Companies, Trucking – Local/Long Distance, Delivery Services – Local",0


In [85]:
nodes_result

step,target,count,route
0,Drug Stores and Pharmacies,191,"[people1012, people1014, people1019, people102..."
1,Fast Food Restaurants,75,"[people1002, people1021, people1032, people106..."
2,Bakeries,41,"[people1025, people1038, people1040, people108..."
3,"Furniture, Home Furnishings, and Equipment Sto...",41,"[people0, people1, people1037, people1099, peo..."
4,"Grocery Stores, Supermarkets",32,"[people1049, people1093, people11, people1151,..."


In [67]:
%%time
startTime = pd.Timestamp("2020-04-30 10:00:00")
endTime = pd.Timestamp("2020-04-30 18:00:00")
time_interval_limit = pd.offsets.Minute(100)
sample_last_step_list = ["Bakeries", "Book Stores"]
QuerySingleNode(startTime, endTime, "Bakeries", -2, time_interval_limit, sample_last_step_list, 5)

CPU times: user 96.9 ms, sys: 2.49 ms, total: 99.4 ms
Wall time: 98.4 ms


step,source,target,count,route,atv
0,Book Stores,Fast Food Restaurants,11.0,"[people2587, people3052, people3135, people377...",39.363636
1,Book Stores,Book Stores,6.0,"[people1616, people3649, people4895, people513...",43.833333
2,Book Stores,"Motor Freight Carriers, Moving and Storage Com...",5.0,"[people2132, people3056, people342, people384,...",35.800000
3,Book Stores,Drug Stores and Pharmacies,1.0,[people6197],46.000000
4,Book Stores,"Grocery Stores, Supermarkets",1.0,[people1427],55.000000
